# Installing dependencies

In [1]:
import sys

!{sys.executable} -m pip install -U dm-sonnet==1.23
!{sys.executable} -m pip install --upgrade tfp-nightly
!{sys.executable} -m pip install tf-nightly
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

Requirement already up-to-date: dm-sonnet==1.23 in /Users/raouldias/anaconda3/lib/python3.7/site-packages (1.23)
You should consider upgrading via the '/Users/raouldias/anaconda3/bin/python -m pip install --upgrade pip' command.
Requirement already up-to-date: tfp-nightly in /Users/raouldias/anaconda3/lib/python3.7/site-packages (0.12.0.dev20200819)
You should consider upgrading via the '/Users/raouldias/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/raouldias/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/raouldias/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/raouldias/anaconda3/bin/python -m pip install --upgrade pip' command.


In [20]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Imports

In [21]:
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import datetime

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Input
from tensorflow_probability import distributions as tfd

# Loading Dataset

In [22]:
DATAPATH = '/Users/raouldias/Desktop/Masters Project/Datasets/brenda_hierarchical.csv'
dataframe = pd.read_csv(DATAPATH, delimiter=',', header=None, skiprows=1, names= ['ec_num', 'organism', 'km_value','substrate', 'comments'])

# Pre-Processing

In [23]:
dataframe = dataframe.drop(columns= ['comments'])
dataframe.drop_duplicates(keep='first', inplace= True)


# Splitting the data into training, testing and validation sets

In [24]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'testing examples')

10889 training examples
2723 validation examples
3404 testing examples


# Seperating labels from data

In [25]:
train_x = train[['ec_num', 'organism', 'substrate']].copy()
test_x = test[['ec_num', 'organism', 'substrate']].copy()
train_y = train['km_value'].copy()
test_y = test['km_value'].copy()

# Functions to map categorical variables to integers

In [26]:
def categorify(df, cat_vars):
    categories = {}
    for cat in cat_vars:
        df[cat] = df[cat].astype("category").cat.as_ordered()
        categories[cat] = df[cat].cat.categories
    return categories

def apply_test(test,categories):
    for cat, index in categories.items():
        test[cat] = pd.Categorical(test[cat],categories=categories[cat],ordered=True)

# Function to produce optimal embedding layer size

In [27]:
def get_emb_sz(cat_col,categories_dict):
    num_classes = len(categories_dict[cat_col])
    return int(min(600,round(1.6*num_classes**0.56)))

# Function to unpack output of Mixture Density Network

In [28]:
def slice_parameter_vectors(parameter_vector):
    """ Returns an unpacked list of paramter vectors.
    """
    print(parameter_vector)
    alpha = parameter_vector[:, :, :c]
    mu = parameter_vector[:, :, c:c*2]
    sigma = parameter_vector[:, :, c*2:c*3]
    
    return alpha,mu,sigma

# Custom loss function to compute the negative log-likelihood of y given the mixture parameters

In [29]:
def gnll_loss(y, parameter_vector):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """
    alpha, mu, sigma = slice_parameter_vectors(parameter_vector) #Unpack parameter vectors
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,       
            scale=sigma))
    
    log_likelihood = gm.log_prob(tf.transpose(y))                 # Evaluate log-probability of y
    
    return -tf.reduce_mean(log_likelihood, axis=-1) 

# Non-negative Exponential Linear Unit  activation function

In [30]:
def nnelu(input):
    """ Computes the Non-Negative Exponential Linear Unit
    """
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

tf.keras.utils.get_custom_objects().update({'nnelu': tf.keras.layers.Activation(nnelu)})

# Coverting the categorical variables to integers

In [31]:
cat_vars = ['ec_num', 'organism', 'substrate']
categories = categorify(train_x, cat_vars)
apply_test(test_x,categories)
for cat in cat_vars:
    train_x[cat] = train_x[cat].cat.codes+1
    test_x[cat] = test_x[cat].cat.codes+1

# Construction of Mixture Density Network

In [37]:
c = 4
no_parameters = 3
num_ec = len(dataframe.ec_num.unique())
num_organism = len(dataframe.organism.unique())
num_substrate = len(dataframe.substrate.unique())

ec_input = Input(shape= (1,),name = 'ec')
organism_input = Input(shape= (1,), name = 'organism')
substrate_input = Input(shape= (1,), name = 'substrate')

ec_features = layers.Embedding(num_ec, get_emb_sz('ec_num', categories))(ec_input)
organism_features = layers.Embedding(num_organism ,get_emb_sz('organism', categories))(organism_input)
substrate_features = layers.Embedding(num_substrate,get_emb_sz('substrate', categories))(substrate_input)

x = layers.concatenate([ec_features, organism_features, substrate_features])

h1 = layers.Dense(100, activation='relu', name='h1')(x)
h2 = layers.Dense(100, activation='relu', name='h2')(h1)

alphas = layers.Dense(c, activation="softmax", name="alphas")(h2)   # Create vector for alpha (softmax constrained)
mus = layers.Dense(c, name="mus")(h2)                               # Create vector for mus
sigmas = layers.Dense(c, activation="nnelu", name="sigmas")(h2)     # Create vector sigmas (nnelu constrained)
pvector = layers.concatenate([alphas, mus, sigmas], name='output') 

model = tf.keras.Model([ec_input, organism_input, substrate_input], pvector)

model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ec (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
organism (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
substrate (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 69)        61617       ec[0][0]                         
_______________________________________________________________________________________

In [38]:
model.compile(loss=gnll_loss, optimizer = "Adam")

# Preparing input for model

In [39]:
input_dict = {'ec': train_x['ec_num'].values,
             'organism': train_x['organism'].values,
             'substrate': train_x['substrate'].values}

# Logging for tensorboard

In [40]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

In [41]:
model.fit(input_dict, train_y, epochs=500, callbacks=[tensorboard_callback])

Epoch 1/500
Tensor("functional_5/output/concat:0", shape=(None, 1, 12), dtype=float32)
Tensor("functional_5/output/concat:0", shape=(None, 1, 12), dtype=float32)
341/341 [==============================] - 2s 5ms/step - loss: 790.2872
Epoch 2/500
341/341 [==============================] - 2s 5ms/step - loss: 43.8260
Epoch 3/500
341/341 [==============================] - 2s 5ms/step - loss: 12.8765
Epoch 4/500
341/341 [==============================] - 2s 5ms/step - loss: 7.6351
Epoch 5/500
341/341 [==============================] - 2s 5ms/step - loss: 5.5667
Epoch 6/500
341/341 [==============================] - 2s 5ms/step - loss: 4.4074
Epoch 7/500
341/341 [==============================] - 2s 5ms/step - loss: 3.6867
Epoch 8/500
341/341 [==============================] - 2s 5ms/step - loss: 3.0986
Epoch 9/500
341/341 [==============================] - 2s 5ms/step - loss: 2.9381
Epoch 10/500
341/341 [==============================] - 2s 6ms/step - loss: 2.7847
Epoch 11/500
341/341 [===

341/341 [==============================] - 2s 5ms/step - loss: 1.8942
Epoch 96/500
341/341 [==============================] - 2s 6ms/step - loss: 1.8966
Epoch 97/500
341/341 [==============================] - 2s 6ms/step - loss: 1.9035
Epoch 98/500
341/341 [==============================] - 2s 6ms/step - loss: 1.8753
Epoch 99/500
341/341 [==============================] - 2s 7ms/step - loss: 1.8972
Epoch 100/500
341/341 [==============================] - 2s 7ms/step - loss: 1.8904
Epoch 101/500
341/341 [==============================] - 2s 7ms/step - loss: 1.8774
Epoch 102/500
341/341 [==============================] - 3s 8ms/step - loss: 1.8952
Epoch 103/500
341/341 [==============================] - 3s 8ms/step - loss: 1.8889
Epoch 104/500
341/341 [==============================] - 3s 8ms/step - loss: 1.8918
Epoch 105/500
341/341 [==============================] - 3s 9ms/step - loss: 1.8842
Epoch 106/500
341/341 [==============================] - 3s 9ms/step - loss: 1.8849
Epoch 107/

341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 196/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 197/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 198/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 199/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 200/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 201/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 202/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 203/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 204/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 205/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 206/500
341/341 [==============================] - 2s 5ms/step - loss: nan
Epoch 207/500
341/341 [===================

KeyboardInterrupt: 

In [19]:
%tensorboard --logdir logs/fit